# SPACESHIP TITANIC
First, I will import the libraries and data needed for analysis.

In [ ]:
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
# Load the training dataset (saving the test dataset for final submission)
df_train = pd.read_csv('./data/train.csv')
df_train.info()

In [ ]:
# Count and enumerate unique categorical values
for col in df_train.columns:
    column = df_train[col]
    if column.dtype in ('bool', 'object'):
        uniques = column.unique()
        print(f"{col} ({len(uniques)})")
        print(f"{uniques}\n")

In [ ]:
# Show feature correlation heatmap
plt.figure(figsize=(15, 5), dpi=150)
sns.heatmap(df_train.corr(), annot=True, cmap='cubehelix')
plt.show()

# PLAN OF ATTACK
For this analysis, I want to see if there is a performance difference between the predictions on the normal feature space and predictions on a principal component latent space.
<br>
##### **Principal Component Analysis**
There are some binary categorical features, and there are some nominal categorical features. If I use one-hot encoding, then I'll significantly expand the dimensionality, especially with features like `Name` and `Cabin`. However, I wonder if running PCA for dimensionality reduction will prove insightful, possibly by finding otherwise-hidden patterns or clusters in the latent space.
<br>
##### **Artificial Neural Network**
I'll use Keras to build a low-depth neural net, tweaking the parameters (architecture) and hyperparameters (Adam parameters) until the model is complex enough not to underfit to the data and regularized enough not to overfit to the data. Then, I will compare the performance of the neural net on the normal feature space and the PCA latent space.

# FEATURE ENGINEERING: Trial 1
I will fully expand every feature through dummy encoding, then I'll re-reduce dimensionality with PCA. Predictions will be on the latent space. I am concerned that the SVD algorithm won't be able to converge due to the very high dimensionality I'm anticipating.

In [ ]:
# Extract features and labels from the training data
X = df_train.iloc[:, :-1]
y = df_train.iloc[:, -1:]
print(f"{X.shape=}")
print(f"{y.shape=}")
X.head()

In [ ]:
# Expand all features
X_clean = pd.get_dummies(X)
X_clean.head()

In [ ]:
# Standardize the features about the origin
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)
X_scaled = pd.DataFrame(X_scaled, columns=X_clean.columns)
X_scaled[X_scaled.isna()] = 0
X_scaled.head()

In [ ]:
# Binarize boolean labels
y_clean = np.zeros(y.shape) + y.to_numpy()
y_clean = pd.DataFrame(y_clean, dtype='uint8')
y_clean.head()

In [ ]:
# Split 20% of data for validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_clean, test_size=0.2, shuffle=True, random_state=0)

print(f"{X_train.shape=}")
print(f"{y_train.shape=}")
print(f"{X_val.shape=}")
print(f"{y_val.shape=}")

In [ ]:
# Code below is retroactively commented out because it doesn't converge.

# Decompose into principal components
# pca = PCA(random_state=0).fit(X_train)
# X_train_pca = pca.transform(X_train)
#
# pca_val = PCA(random_state=0).fit(X_val)
# X_val_pca = pca.transform(X_val)
#
# print(f"{X_train_pca.shape=}")
# print(f"{X_val_pca.shape=}")

# FEATURE ENGINEERING: Trial 2
Indeed, the SVD algorithm did not converge because of the dimensionality. To prevent this issue, I'll conduct this trial the same as Trial 1, except I will be more intentional when encoding categorical data instead of just naively expanding it.

In [ ]:
# Clean up the features

# `Name` and `PassengerId` are unique identifiers, probably not highly correlated to the target
X_clean = X.drop(['Name', 'PassengerId'], axis=1)

# Only dummy encode features with fewer than 5 categories
expand_cols = ['HomePlanet', 'Destination']
onehot_cols = pd.get_dummies(X_clean[expand_cols])
X_clean = X_clean.drop(expand_cols, axis=1)
X_clean = X_clean.join(onehot_cols)

# Binarize boolean features
for col in ('CryoSleep', 'VIP'):
    column = X_clean[col].copy()
    column[column.isna()] = 0
    column = np.zeros(column.shape) + column.to_numpy()
    column = pd.Series(column, dtype='uint8')
    X_clean[col] = column

# Split `Cabin` encoding by splitting it into 3 features
cabins = {'Cabin1': [],
          'Cabin2': [],
          'Cabin3': []}

for i, row in X_clean.copy().iterrows():
    if type(row['Cabin']) == str:
        c1, c2, c3 = row['Cabin'].split('/')
        cabins['Cabin1'].append(c1)
        cabins['Cabin2'].append(c2)
        cabins['Cabin3'].append(c3)

X_clean = X_clean.drop('Cabin', axis=1)
X_clean = X_clean.join(pd.DataFrame(cabins))

expand_cabins = ['Cabin1', 'Cabin2', 'Cabin3']
onehot_cabins = pd.get_dummies(X_clean[expand_cabins])
X_clean = X_clean.drop(expand_cabins, axis=1)
X_clean = X_clean.join(onehot_cabins)

X_clean.head()

In [ ]:
# Normalize features between 0 and 1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_clean)
X_scaled = pd.DataFrame(X_scaled, columns=X_clean.columns)
X_scaled[X_scaled.isna()] = 0
X_scaled.head()

In [ ]:
# Split 20% of data for validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_clean, test_size=0.2, shuffle=True, random_state=0)

print(f"{X_train.shape=}")
print(f"{y_train.shape=}")
print(f"{X_val.shape=}")
print(f"{y_val.shape=}")

In [ ]:
# Decompose into principal components
pca = PCA(random_state=0).fit(X_train)
X_train_pca = pca.transform(X_train)

pca_val = PCA(random_state=0).fit(X_val)
X_val_pca = pca.transform(X_val)

print(f"{X_train_pca.shape=}")
print(f"{X_val_pca.shape=}")

In [ ]:
# Visualize cumulative explained variance
exp_var = np.array([sum(pca.explained_variance_ratio_[:(i + 1)]) for i in range(pca.n_components_)])
exp_var_val = np.array([sum(pca_val.explained_variance_ratio_[:(i + 1)]) for i in range(pca_val.n_components_)])

plt.style.use('seaborn-dark')
plt.figure(figsize=(15, 5), dpi=150)
plt.plot(exp_var, label="Training Data Variance")
plt.plot(exp_var_val, label="Validation Data Variance")

plt.title("Cumulative explained variance within the first principal components")
plt.xlabel("Number of principal components")
plt.ylabel("Variance explained")
plt.xticks([_ for _ in np.arange(0, 2000, 100)])
plt.yticks([_ for _ in np.arange(0, 1.1, .1)])
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
# Plot data projections
plt.figure(figsize=(15, 15), dpi=150)
plt.suptitle("First eight principal components, colored by target label")

plt.subplot(221)
plt.xlabel("PC1")
plt.ylabel("PC2")
sns.scatterplot(x=X_train_pca @ pca.components_[0],
                y=X_train_pca @ pca.components_[1],
                hue=y_train[0])

plt.subplot(222)
plt.xlabel("PC3")
plt.ylabel("PC4")
sns.scatterplot(x=X_train_pca @ pca.components_[2],
                y=X_train_pca @ pca.components_[3],
                hue=y_train[0])

plt.subplot(223)
plt.xlabel("PC5")
plt.ylabel("PC6")
sns.scatterplot(x=X_train_pca @ pca.components_[4],
                y=X_train_pca @ pca.components_[5],
                hue=y_train[0])

plt.subplot(224)
plt.xlabel("PC7")
plt.ylabel("PC8")
sns.scatterplot(x=X_train_pca @ pca.components_[6],
                y=X_train_pca @ pca.components_[7],
                hue=y_train[0])

plt.show()

After looking at the scatter plots above colored by target label, I don't have high hopes that PCA will give me any better clue for classification.

# MODEL SELECTION
Defining some convenience methods here.

In [ ]:
def build_model(layers:     List[tf.keras.layers.Layer],
                optimizer:  tf.keras.optimizers.Optimizer | str,
                loss:       tf.keras.losses.Loss | str,
                metrics:    List[str]) -> tf.keras.models.Model:
    """Constructs and compiles a Keras model, then prints summary."""

    model = Sequential(layers)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    model.summary()

    return model

In [ ]:
def plot_history(hist1, hist2):
    """Visualizes training performance over two spaces."""
    performance_1 = pd.DataFrame(hist1.history)
    performance_2 = pd.DataFrame(hist2.history)

    plt.figure(figsize=(15, 5), dpi=150)
    plt.suptitle('Training Performance')

    # feature space
    plt.subplot(121)
    plt.title("History 1")
    plt.plot(performance_1['accuracy'], label='Training')
    plt.plot(performance_1['val_accuracy'], label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim(0.5, 1)
    plt.legend(loc='lower right')

    # pca latent space
    plt.subplot(122)
    plt.title("History 2")
    plt.plot(performance_2['accuracy'], label='Training')
    plt.plot(performance_2['val_accuracy'], label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim(0.5, 1)
    plt.legend(loc='lower right')

    plt.show()

# MODEL SELECTION: Trial 1
Now that I have the feature space `X_train`/`X_val` and the PCA latent space `X_train_pca`/`X_val_pca`, I will predict over both spaces with a low-depth neural net.

In [ ]:
# Define model
layers_1 = [Dense(100, 'relu', input_shape=(X_train.shape[1],)),
            Dense(200, 'relu'),
            Dense(100, 'relu'),
            Dropout(0.25),

            Dense(200, 'relu'),
            Dense(300, 'relu'),
            Dense(200, 'relu'),
            Dense(100, 'relu'),
            Dropout(0.25),

            Dense(50, 'relu'),
            Dense(25, 'relu'),
            Dense(1, 'sigmoid')]

opt_1 = Adam(learning_rate=0.0005,
             beta_1=0.99,
             beta_2=0.99)

model_1 = build_model(layers_1, opt_1, 'mse', ['accuracy'])

In [ ]:
# Fit model to feature space
history_1 = model_1.fit(X_train, y_train,
                        epochs=100, verbose=1,
                        validation_data=(X_val, y_val))

In [ ]:
# Fit model to PCA latent space
history_pca_1 = model_1.fit(X_train_pca, y_train,
                            epochs=100, verbose=1,
                            validation_data=(X_val_pca, y_val))

In [ ]:
# Visualize training performance over both spaces
plot_history(history_1, history_pca_1)

# MODEL SELECTION: Trial 2
Clearly, the model overfit to the data. I wonder if I went a little heavy on the hidden layers and hidden neurons. I'll simplify the model and test performance again.

In [ ]:
# Define simpler model
layers_2 = [Dense(64, 'relu', input_shape=(X_train.shape[1],)),
            Dense(128, 'relu'),
            Dense(64, 'relu'),
            Dense(1, 'sigmoid')]

opt_2 = Adam(learning_rate=0.0008,
             beta_1=0.98,
             beta_2=0.98)

model_2 = build_model(layers_2, opt_2, 'mse', ['accuracy'])

In [ ]:
# Fit model to feature space
history_2 = model_2.fit(X_train, y_train,
                        epochs=100, verbose=1,
                        validation_data=(X_val, y_val))

In [ ]:
# Fit model to PCA latent space
history_pca_2 = model_2.fit(X_train_pca, y_train,
                            epochs=100, verbose=1,
                            validation_data=(X_val_pca, y_val))

In [ ]:
# Visualize training performance over both spaces
plot_history(history_2, history_pca_2)

# FEATURE ENGINEERING: Trial 3
Even with a simpler model, it still heavily overfit to the data. Also, in both trials, the neural network performance about the same on the feature space and the latent space. This is possibly because I'm not dropping any principal components, so the PCA space embedding and the normal feature embedding can be represented as linear transformations of each other, which the neural network may have learned.
<br>
With these observations in mind, I will forgo the effort for principal component decomposition. Instead, I'll focus on encoding categorical features more efficiently so that their information can be extracted better.

In [ ]:
# Clean up the features

# `Name` and `PassengerId` are unique identifiers, probably not highly correlated to the target
X_clean = X.drop(['Name', 'PassengerId'], axis=1)

# Only dummy encode features with fewer than 5 categories
expand_cols = ['HomePlanet', 'Destination']
onehot_cols = pd.get_dummies(X_clean[expand_cols])
X_clean = X_clean.drop(expand_cols, axis=1)
X_clean = X_clean.join(onehot_cols)

# Binarize boolean features
for col in ('CryoSleep', 'VIP'):
    column = X_clean[col].copy()
    column[column.isna()] = 0
    column = np.zeros(column.shape) + column.to_numpy()
    column = pd.Series(column, dtype='uint8')
    X_clean[col] = column

# Split `Cabin` encoding by splitting it into 3 features
cabins = {'Cabin1': [],
          'Cabin2': [],
          'Cabin3': []}

for i, row in X_clean.copy().iterrows():
    if type(row['Cabin']) == str:
        c1, c2, c3 = row['Cabin'].split('/')

        # Numeralize cabin location by summing ASCII values
        cabins['Cabin1'].append(sum(ord(c) for c in str(c1)))
        cabins['Cabin2'].append(sum(ord(c) for c in str(c2)))
        cabins['Cabin3'].append(sum(ord(c) for c in str(c3)))

# Semanticize cabin location by min-max scaling the ASCII value sums
for i, cabin in cabins.items():
    cabins[i] = np.array(cabin)
    cabins[i] = (cabins[i] - cabins[i].min()) / (cabins[i].max() / cabins[i].min())

X_clean = X_clean.drop('Cabin', axis=1)
X_clean = X_clean.join(pd.DataFrame(cabins))

X_clean.head()

In [ ]:
# Normalize features between 0 and 1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_clean)
X_scaled = pd.DataFrame(X_scaled, columns=X_clean.columns)
X_scaled[X_scaled.isna()] = 0
X_scaled.head()

In [ ]:
# Split 20% of data for validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_clean, test_size=0.2, shuffle=True, random_state=0)

print(f"{X_train.shape=}")
print(f"{y_train.shape=}")
print(f"{X_val.shape=}")
print(f"{y_val.shape=}")

In [ ]:
# Show feature correlation heatmap after preprocessing
plt.figure(figsize=(15, 5), dpi=150)
sns.heatmap(X_train.join(y_train).corr(), annot=True, cmap='cubehelix')
plt.show()

Now that the data is preprocessed, every feature shows up on the heatmap. Subsequently, we can see that `CryoSleep` seems to have a moderate positive correlation with the target, while `RoomService`, `Spa`, and `VRDeck` have a mild negative correlation with the target. Also of note, splitting `Cabin` into three features didn't seem to add to the feature's target correlation.

# FEATURE ENGINEERING: Trial 4

 I will re-engineer the feature set, taking into account the observations made on the heatmap above. I want to try to accentuate the correlations somehow.

In [ ]:
# Clean up the features
X_clean = X.drop(['Name', 'PassengerId', 'Cabin'], axis=1)

# Only dummy encode features with fewer than 5 categories
expand_cols = ['HomePlanet', 'Destination']
onehot_cols = pd.get_dummies(X_clean[expand_cols])
X_clean = X_clean.drop(expand_cols, axis=1)
X_clean = X_clean.join(onehot_cols)

# Binarize boolean features
for col in ('CryoSleep', 'VIP'):
    column = X_clean[col].copy()
    column[column.isna()] = 0
    column = np.zeros(column.shape) + column.to_numpy()
    column = pd.Series(column)
    X_clean[col] = column

X_clean.head()

In [ ]:
# Normalize features between 0 and 1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_clean)
X_scaled = pd.DataFrame(X_scaled, columns=X_clean.columns)
X_scaled[X_scaled.isna()] = 0
X_scaled.head()

In [ ]:
# Accentuate correlations through logistic expansion (experimental)
def logitistic_expansion(a: np.ndarray, magic: int) -> np.ndarray:
    """I'm making this up, but here's the objective:

    1. Take a range of data that is normalized between 0 and 1
    2. Accentuate the spread by making lower values lower and higher values higher.
    3. See if accentuating the spread improves correlation and model performance.

    Algorithm:
    - Subtract 0.5 from each value
        - Small values will have negative differences
        - Large values will have positive differences
    - Exponentiate the differences
        - Positive differences will create product > 1
        - Negative differences will create product < 1
    - Scale the value by the exponentiated difference
        - Smaller values will get smaller
        - Larger values will get larger

    Magic is the number of times the accentuation will occur.
    """
    return a * (np.e ** (a - 0.5)) ** magic

X_expanded = logitistic_expansion(X_scaled, 42)

In [ ]:
# Split 20% of data for validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_clean, test_size=0.2, shuffle=True, random_state=0)
X_train_exp, X_val_exp, _, _ = train_test_split(X_expanded, y_clean, test_size=0.2, shuffle=True, random_state=0)

print(f"{X_train.shape=}")
print(f"{X_train_exp.shape=}")
print(f"{y_train.shape=}")
print(f"{X_val.shape=}")
print(f"{X_val_exp.shape=}")
print(f"{y_val.shape=}")

In [ ]:
# Show feature correlation heatmap after preprocessing
plt.figure(figsize=(15, 5), dpi=150)
sns.heatmap(X_train_exp.join(y_train).corr(), annot=True, cmap='cubehelix')
plt.show()

# MODEL SELECTION: Trial 3
I can see from the heatmap that the expansion did not work as intended. I predict that the model will fail miserably on the expanded data, but we will try anyway, for science.

In [ ]:
# Define another model
layers_3 = [Dense(64, 'relu', input_shape=(X_train.shape[1],)),
            Dense(128, 'relu'),
            Dense(64, 'relu'),
            Dense(1, 'sigmoid')]

opt_3 = Adam(learning_rate=0.0008,
             beta_1=0.98,
             beta_2=0.98)

model_3 = build_model(layers_3, opt_3, 'mse', ['accuracy'])

In [ ]:
# Fit model to feature space
history_3 = model_3.fit(X_train, y_train,
                        epochs=100, verbose=1,
                        validation_data=(X_val, y_val))

In [ ]:
# Fit model to feature space
history_3_exp = model_3.fit(X_train_exp, y_train,
                            epochs=100, verbose=1,
                            validation_data=(X_val_exp, y_val))

In [ ]:
# Visualize training performance
plot_history(history_3, history_3_exp)

As predicted, model performance on the expanded data did very poorly. However, it is interesting to note that after dropping the `Cabin` feature entirely, the performance of the model on the data after only scaling is improved from before.

# FEATURE ENGINEERING: Trial 5
Seeing how dropping the `Cabin`, which had close-to-zero correlation, improved model performance, I will drop all of the other "neutral" features, hoping that it will reduce noise in the model's training and improve performance.

In [ ]:
# Clean up the features
X_clean = X.drop(['Name', 'PassengerId', 'Cabin', 'Age', 'VIP', 'FoodCourt', 'ShoppingMall'], axis=1)

# Only dummy encode features with small (<5) expansion
expand_cols = ['HomePlanet', 'Destination']
onehot_cols = pd.get_dummies(X_clean[expand_cols])
X_clean = X_clean.drop(expand_cols, axis=1)
X_clean = X_clean.join(onehot_cols)

# Binarize `CryoSleep`
cryo = X_clean['CryoSleep'].copy()
cryo[cryo.isna()] = 0
cryo = np.zeros(cryo.shape) + cryo.to_numpy()
cryo = pd.Series(cryo)
X_clean['CryoSleep'] = cryo

X_clean.head()

In [ ]:
# Normalize features between 0 and 1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_clean)
X_scaled = pd.DataFrame(X_scaled, columns=X_clean.columns)
X_scaled[X_scaled.isna()] = 0
X_scaled.head()

In [ ]:
# Split 20% of data for validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_clean, test_size=0.2, shuffle=True, random_state=0)

print(f"{X_train.shape=}")
print(f"{y_train.shape=}")
print(f"{X_val.shape=}")
print(f"{y_val.shape=}")

In [ ]:
# Show feature correlation heatmap after preprocessing
plt.figure(figsize=(15, 5), dpi=150)
sns.heatmap(X_train.join(y_train).corr(), annot=True, cmap='cubehelix')
plt.show()

# MODEL SELECTION: Trial 4
Now that only features with a non-neutral correlation factor are in the feature set, I want to see how performance is affected.

In [ ]:
# Define another model
layers_4 = [Dense(64, 'relu', input_shape=(X_train.shape[1],)),
            Dense(128, 'relu'),
            Dense(64, 'relu'),
            Dense(1, 'sigmoid')]

opt_4 = Adam(learning_rate=0.0008,
             beta_1=0.98,
             beta_2=0.98)

model_4 = build_model(layers_4, opt_4, 'mse', ['accuracy'])

In [ ]:
# Fit model to feature space
history_4 = model_4.fit(X_train, y_train,
                        epochs=100, verbose=1,
                        validation_data=(X_val, y_val))

In [ ]:
# Visualize training performance
performance = pd.DataFrame(history_4.history)

plt.figure(figsize=(15, 5), dpi=150)
plt.plot(performance['accuracy'], label='Training')
plt.plot(performance['val_accuracy'], label='Validation')

plt.title("Training Performance")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim(0.5, 1)

plt.legend(loc='lower right')
plt.grid()
plt.show()

The model performance did improve, but it seems to have leveled out just under 80% accuracy.

# MODEL SELECTION: Trial 5
I will try to tune the model architecture and the optimizer to improve performance.

In [ ]:
# Define another model
layers_5 = [Dense(64, 'relu', input_shape=(X_train.shape[1],)),
            Dense(128, 'relu'),
            Dropout(0.1),

            Dense(128, 'relu'),
            Dense(128, 'relu'),
            Dropout(0.1),

            Dense(128, 'relu'),
            Dense(128, 'relu'),
            Dropout(0.1),

            Dense(64, 'relu'),
            Dense(1, 'sigmoid')]

model_5 = build_model(layers_5, 'adam', 'mse', ['accuracy'])

In [ ]:
# Fit model to feature space
history_5 = model_5.fit(X_train, y_train,
                        epochs=100, verbose=1,
                        validation_data=(X_val, y_val))

In [ ]:
# Visualize training performance
performance = pd.DataFrame(history_5.history)

plt.figure(figsize=(15, 5), dpi=150)
plt.plot(performance['accuracy'], label='Training')
plt.plot(performance['val_accuracy'], label='Validation')

plt.title("Training Performance")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim(0.5, 1)

plt.legend(loc='lower right')
plt.grid()
plt.show()

# CONCLUSION

Through this analysis, I've learned a few key things:
- PCA doesn't magically make any dataset linearly separable.
- Having way too many features will cause the model to overfit quickly.
- Accentuating separation of scaled values actually compresses correlation.
- Removing features with close-to-zero target correlation can reduce noise and improve performance.

# SUBMISSION
Now that I've reached the end of the analysis and model selection, I will now predict on the test data and submit to Kaggle.

In [ ]:
# Load the test dataset
X_test = pd.read_csv('./data/test.csv')
X_test.info()

In [ ]:
# Clean up the features
X_test_clean = X_test.drop(['Name', 'PassengerId', 'Cabin', 'Age', 'VIP', 'FoodCourt', 'ShoppingMall'], axis=1)

# Only dummy encode features with fewer than 5 categories
expand_cols = ['HomePlanet', 'Destination']
onehot_cols = pd.get_dummies(X_test_clean[expand_cols])
X_test_clean = X_test_clean.drop(expand_cols, axis=1)
X_test_clean = X_test_clean.join(onehot_cols)

# Binarize `CryoSleep`
cryo = X_test_clean['CryoSleep'].copy()
cryo[cryo.isna()] = 0
cryo = np.zeros(cryo.shape) + cryo.to_numpy()
cryo = pd.Series(cryo)
X_test_clean['CryoSleep'] = cryo

X_test_clean.head()

In [ ]:
# Normalize features between 0 and 1
scaler = MinMaxScaler()
X_test_scaled = scaler.fit_transform(X_test_clean)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_clean.columns)
X_test_scaled[X_test_scaled.isna()] = 0
X_test_scaled.head()

In [ ]:
prediction = model_5.predict(X_test_scaled)
prediction

In [ ]:
submission = pd.DataFrame(X_test['PassengerId']).join(pd.DataFrame(prediction))
submission.columns = ['PassengerId', 'Transported']
submission['Transported'] = submission['Transported'].apply(lambda x: True if x >= 0.5 else False)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)

# FINAL RESULT
I placed 1366/2057 with an accuracy score of 0.78372!